# Metro's of the world

This notebook has a few examples of pulling Metro (Underground/Subway) networks from OpenStreetMap (OSM) and plotting them in Bokeh plots. The idea is to produce something interactive and fun to play with. In the future I might try combining the metros in a single plot for comparisons, e.g. size, stops, lines etc. For example like [this](https://www.google.com/url?sa=i&url=https%3A%2F%2Fsoranews24.com%2F2015%2F03%2F23%2Fwe-finns-just-like-it-simple-net-users-cant-get-enough-of-helsinki-metro-map%2F&psig=AOvVaw1j5fY74y3ffxBZ4UK0NaJs&ust=1628545765978000&source=images&cd=vfe&ved=0CAoQjRxqFwoTCODVjfWzovICFQAAAAAdAAAAABAD])

The code is not standardised, and I've tried a few different ways of getting the required data from OSM. For example creating a bounding box (area) and extracting subway tags, or alternatively, selecting the Metro relationship for the network and then pulling data from its members. The second approach is slower to run, but is generally more comprehensive, however, stations and lines are then a single colour and repeated for other lines on the network, rather than being combined into a single station or track.

As well as standardising a little, e.g. turning into functions, it would be good to seperate out the data, so that each metro is stored separately, rather than overwriting, making it easier to combine at a later stage.

## Setup

Import libraries and standard tools used

In [1]:
import os
import requests
import json

import openstreetmap_mapping as osm

import pandas as pd

from bokeh.plotting import show, output_file
from bokeh.io import output_notebook

from bokeh.plotting import figure
from bokeh.models import WMTSTileSource, ColumnDataSource
from bokeh.models import HoverTool

from pyproj import Transformer

output_notebook()

Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
area_Glasgow = "(55.82, -4.34, 55.9, -4.22)"
area_London = "(51.3426, -0.5527, 51.6726, 0.1929)"
area_Moscow = "(55.503, 37.0789,56.003, 38.1871)"
area_Budapest = "(47.4439, 18.9315, 47.5851, 19.1711)"
area_Paris = "(48.744, 2.1265, 49.0316, 2.6099)"

In [4]:
MAP_TILES = {"OpenMap": WMTSTileSource(url="http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png"),
         "ESRI": WMTSTileSource(url="https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{Z}/{Y}/{X}.jpg"),
         "OpenTopoMap": WMTSTileSource(url="https://tile.opentopomap.org/{Z}/{X}/{Y}.png")}

In [5]:
# Use pyproj to transform longitude and latitude into web-mercator and add to a copy of the asset dataframe
TRANSFORM_4326_TO_3857 = Transformer.from_crs("EPSG:4326", "EPSG:3857")
 

In [6]:
# Place to save data so that the number of OSM requests are reduced when re-loading the same data
path_save = r"data/metro/"

if not os.path.exists(path_save):
    os.makedirs(path_save)
    

In [28]:
def load_metro(member_id=10713004,path_save=r"data/metro/",name="paris"):

    # load the whole metro relation
    if os.path.isfile(path_save+"df_"+name+"_metro.pkl"):
        df_paris_metro = pd.read_pickle(path_save+"df_"+name+"_metro.pkl")
    else:
        df_paris_metro = osm.toolkit.get_osm_data_by_id(member_id=member_id,obj_type="rel")
        df_paris_metro.to_pickle(path_save+"df_"+name+"_metro.pkl")

        
    # load the metro map
    if os.path.isfile(path_save+"df_"+name+"_railways.pkl"):
        df_paris_railways = pd.read_pickle(path_save+"df_"+name+"_railways.pkl")
    else:
        df_paris_railways = pd.DataFrame(columns=["id"])

        
    # load the lines and underlying members
    df_paris_metro_lines = pd.DataFrame.from_records(df_paris_metro["members"][0])

    for metro_line_id in df_paris_metro_lines["ref"]:

        # if metro_line_id not in 
        
        #print("line_id " + str(metro_line_id))

        df_metro_line = osm.toolkit.get_osm_data_by_id(member_id=metro_line_id,obj_type="rel")

        df_metro_routes = pd.DataFrame.from_records(df_metro_line["members"][0])

        for route in df_metro_routes["ref"]:

            #print(route)

            if route not in df_paris_railways["id"].values:

                #print(route)

                df_metro_route = osm.toolkit.get_osm_data_by_id(member_id=route,obj_type="rel",output="geom")

                if not df_metro_route.empty:

                    members = pd.DataFrame.from_records(df_metro_route.iloc[0]["members"])
                    rails = members[members["role"]==""]
                    stations = members[members["role"]=="station"]
                    stops = members[members["role"]=="stop"]

                    if not rails.empty:            
                        df_metro_route["rails"] = [rails["geometry"]]

                    if not stations.empty:
                        df_metro_route["stations"] = [stations[["ref","lat","lon"]]]

                    if not stops.empty:
                        stop_ids = str(list(stops["ref"])).replace("[","").replace("]","")

                        stops_info = osm.toolkit.get_osm_data_by_id(member_id="id:"+stop_ids,obj_type="node")  

                        df_metro_route["stops"] = [stops_info]

                    df_paris_railways = pd.concat([df_paris_railways,df_metro_route])

                    df_paris_railways.to_pickle(path_save+"df_"+name+"_railways.pkl")
                    
    return df_paris_railways


In [29]:
def create_metro_map(df_railways,output_file_name="Paris_metro.html"):
    
    # create a plot of the subway system    
    output_file(output_file_name)

    p = figure(plot_width=800, plot_height=800,
                        x_axis_type="mercator", y_axis_type="mercator")

    p.add_tile(MAP_TILES['ESRI'])

    tooltips = [("id", "@id"),
                ("name", "@name"),
                ("line", "@line"),
                ("network", "@network"),
                ("operator", "@operator"),
                ("(lat,lon)", "@coordinates")]

    stations = list()

    # plot the railway lines    
    for cnt,line in df_railways.iterrows():

        #print(line['name'])

        xx = list()
        yy = list()

        for way in line['rails'].dropna():


            xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(way)['lat']),
                                                     list(pd.DataFrame.from_records(way)['lon']))

            xx.append(xxx)
            yy.append(yyy)


        source = ColumnDataSource({'xs':xx,'ys':yy})

        p.multi_line('xs',
               'ys',
               source=source,
               line_width=2,
               legend_label=line['short_name'],
               line_color = line['colour'],
              )


        line['stops']['xs'],line['stops']['ys'] = TRANSFORM_4326_TO_3857.transform(list(line['stops']['lat']),
                                                     list(line['stops']['lon']))

        line['stops']['coordinates'] = tuple(zip(line['stops']["lat"],line['stops']["lon"]))

        line['stops']['line'] = line['short_name']
        line['stops']['network'] = line['network']
        line['stops']['operator'] = line['operator']

        source = ColumnDataSource(line['stops'])

        station = p.circle(x="xs", y="ys",
                                source=source,
                                radius=100,
                                line_width=1,
                                fill_color = line['colour'],
                                line_color = 'white',
                                legend_label=line['short_name']
                                )


        stations.append(station)


    p.add_tools(HoverTool(tooltips=tooltips, renderers=stations))

    p.legend.click_policy="hide"


    show(p)    
    
    return p

## Glasgow's Clockwork Orange

## Moscow Metro

In [ ]:
df_moscow_railways = load_metro(member_id=10713004,path_save=r"data/metro/",name="paris")

In [ ]:
moscow_metro_map = create_metro_map(df_railways=df_moscow_railways,output_file_name="Paris_metro.html")

## Budapest Metro

## Paris Metro

In [30]:
df_paris_railways = load_metro(member_id=10713004,path_save=r"data/metro/",name="paris")

No specified nodes found in area
No specified nodes found in area
No specified nodes found in area
No specified nodes found in area


In [31]:
paris_metro_map = create_metro_map(df_railways=df_paris_railways,output_file_name="Paris_metro.html")

## London Underground

Using relatives rather than line tags

In [ ]:
df_london_railways = load_metro(member_id=7225135,path_save=r"data/metro/",name="london")

In [ ]:
london_metro_map = create_metro_map(df_railways=df_london_railways,output_file_name="London_metro.html")

In [ ]:
df_railways = pd.DataFrame()

df_metro = osm.toolkit.get_osm_data_by_id(member_id=7225135,obj_type="rel") #London metro OSM ID 7225135

df_metro_lines = pd.DataFrame.from_records(df_metro['members'][0])

for metro_line_id in df_metro_lines['ref']:
    
    #print(metro_line_id)
    
    df_metro_line = osm.toolkit.get_osm_data_by_id(member_id=metro_line_id,obj_type="rel")
    
    #print(df_metro_line.iloc[0]['name'])
    
    df_metro_routes = pd.DataFrame.from_records(df_metro_line['members'][0])
    
    for route in df_metro_routes['ref']:
        
        df_metro_route = osm.toolkit.get_osm_data_by_id(member_id=route,obj_type="rel",output="geom")
    
        if not df_metro_route.empty:
            
            members = pd.DataFrame.from_records(df_metro_route.iloc[0]['members'])
            rails = members[members['role']==""]
            stations = members[members['role']=="station"]
            stops = members[(members['role']=="stop") | (members['role']=="stop_entry_only") | (members['role']=="stop_exit_only")]

            if not rails.empty:            
                df_metro_route['rails'] = [rails['geometry']]

            if not stations.empty:
                df_metro_route['stations'] = [stations[['ref','lat','lon']]]

            if not stops.empty:
                stop_ids = str(list(stops['ref'])).replace('[','').replace(']','')
                stops_info = osm.toolkit.get_osm_data_by_id(member_id="id:"+stop_ids,obj_type="node")  
                df_metro_route['stops'] = [stops_info]

            df_railways = pd.concat([df_railways,df_metro_route])
        
        #print(df_railways)
    

In [ ]:
# create a plot of the subway system    
output_file("London_metro.html")

p = figure(plot_width=800, plot_height=800,
                    x_axis_type="mercator", y_axis_type="mercator")

p.add_tile(MAP_TILES['ESRI'])

tooltips = [("id", "@id"),
            ("name", "@name"),
            ("line", "@line"),
            ("network", "@network"),
            ("operator", "@operator"),
            ("(lat,lon)", "@coordinates")]

station_glpyhs = list()

# plot the railway lines    
for cnt,line in df_railways.iterrows():

    #print(line['name'])
    
    xx = list()
    yy = list()
    
    for way in line['rails'].dropna():
    

        xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(way)['lat']),
                                                 list(pd.DataFrame.from_records(way)['lon']))

        xx.append(xxx)
        yy.append(yyy)

        
    source = ColumnDataSource({'xs':xx,'ys':yy})
    
    p.multi_line('xs',
           'ys',
           source=source,
           line_width=2,
           legend_label=line['ref'],
           line_color = line['colour'],
          )

    if isinstance(line['stops'],pd.DataFrame):
    
        line['stops']['xs'],line['stops']['ys'] = TRANSFORM_4326_TO_3857.transform(list(line['stops']['lat']),
                                                     list(line['stops']['lon']))

        line['stops']['coordinates'] = tuple(zip(line['stops']["lat"],line['stops']["lon"]))

        line['stops']['line'] = line['ref']
        line['stops']['network'] = line['network']
        line['stops']['operator'] = line['operator']

        source = ColumnDataSource(line['stops'])

        station = p.circle(x="xs", y="ys",
                                source=source,
                                radius=100,
                                line_width=1,
                                fill_color = line['colour'],
                                line_color = 'white',
                                legend_label=line['ref']
                                )


        station_glpyhs.append(station)


p.add_tools(HoverTool(tooltips=tooltips, renderers=station_glpyhs))

p.legend.click_policy="hide"


show(p)    


## New York Metro

In [ ]:
df_railways = pd.DataFrame()

df_metro = osm.toolkit.get_osm_data_by_id(member_id=2621040,obj_type="rel")

df_metro_lines = pd.DataFrame.from_records(df_metro['members'][0])

for metro_line_id in df_metro_lines['ref']:
    
    #print(metro_line_id)
    
    df_metro_line = osm.toolkit.get_osm_data_by_id(member_id=metro_line_id,obj_type="rel")
    
    #print(df_metro_line.iloc[0]['name'])
    
    df_metro_routes = pd.DataFrame.from_records(df_metro_line['members'][0])
    
    for route in df_metro_routes['ref']:
        
        df_metro_route = osm.toolkit.get_osm_data_by_id(member_id=route,obj_type="rel",output="geom")
    
        if not df_metro_route.empty:
            
            members = pd.DataFrame.from_records(df_metro_route.iloc[0]['members'])
            rails = members[members['role']==""]
            stations = members[members['role']=="station"]
            stops = members[(members['role']=="stop") | (members['role']=="stop_entry_only") | (members['role']=="stop_exit_only")]

            if not rails.empty:            
                df_metro_route['rails'] = [rails['geometry']]

            if not stations.empty:
                df_metro_route['stations'] = [stations[['ref','lat','lon']]]

            if not stops.empty:
                stop_ids = str(list(stops['ref'])).replace('[','').replace(']','')
                stops_info = osm.toolkit.get_osm_data_by_id(member_id="id:"+stop_ids,obj_type="node")  
                df_metro_route['stops'] = [stops_info]

            df_railways = pd.concat([df_railways,df_metro_route])
        
        #print(df_railways)
    

In [ ]:
# create a plot of the subway system    
output_file("NewYork_metro.html")

p = figure(plot_width=800, plot_height=800,
                    x_axis_type="mercator", y_axis_type="mercator")
    
p.add_tile(MAP_TILES["ESRI"])

tooltips = [("id", "@id"),
            ("name", "@name"),
            ("line", "@line"),
            ("network", "@network"),
            ("operator", "@operator"),
            ("(lat,lon)", "@coordinates")]

station_glpyhs = list()

# plot the railway lines    
for cnt,line in df_railways.iterrows():

    #print(line["name"])
    
    xx = list()
    yy = list()
    
    for way in line["rails"].dropna():
    

        xxx,yyy = TRANSFORM_4326_TO_3857.transform(list(pd.DataFrame.from_records(way)["lat"]),
                                                 list(pd.DataFrame.from_records(way)["lon"]))

        xx.append(xxx)
        yy.append(yyy)

        
    source = ColumnDataSource({"xs":xx,"ys":yy})
    
    p.multi_line("xs",
           "ys",
           source=source,
           line_width=2,
           legend_label=line["ref"],
           line_color = line["colour"],
          )

    if isinstance(line["stops"],pd.DataFrame):
    
        line["stops"]["xs"],line["stops"]["ys"] = TRANSFORM_4326_TO_3857.transform(list(line["stops"]["lat"]),
                                                     list(line["stops"]["lon"]))

        line["stops"]["coordinates"] = tuple(zip(line["stops"]["lat"],line["stops"]["lon"]))

        line["stops"]["line"] = line["ref"]
        line["stops"]["network"] = line["network"]
        line["stops"]["operator"] = line["operator"]

        source = ColumnDataSource(line["stops"])

        station = p.circle(x="xs", y="ys",
                                source=source,
                                radius=100,
                                line_width=1,
                                fill_color = line["colour"],
                                line_color = "white",
                                legend_label=line["ref"]
                                )


        station_glpyhs.append(station)


p.add_tools(HoverTool(tooltips=tooltips, renderers=station_glpyhs))

p.legend.click_policy="hide"


show(p)    
